In [2]:
import carla

client = carla.Client('localhost', 2000)
client.set_timeout(10.0)
world = client.get_world()


In [2]:
blueprint_library = world.get_blueprint_library()
vehicle_bp = blueprint_library.find('vehicle.nissan.patrol')
spawn_point = world.get_map().get_spawn_points()[0]
vehicle = world.spawn_actor(vehicle_bp, spawn_point)

# Attach RGB camera
camera_bp = blueprint_library.find('sensor.camera.rgb')
camera_bp.set_attribute('image_size_x', '640')
camera_bp.set_attribute('image_size_y', '480')
camera_bp.set_attribute('fov', '110')

camera_transform = carla.Transform(carla.Location(x=1.5, z=2.4))  # hood position
camera = world.spawn_actor(camera_bp, camera_transform, attach_to=vehicle)


In [3]:
vehicle.set_autopilot(True)  # autopilot ON


In [4]:
import cv2
import numpy as np
import os
import csv

os.makedirs('dataset/images', exist_ok=True)
control_data = open('dataset/controls.csv', mode='w', newline='')
csv_writer = csv.writer(control_data)
csv_writer.writerow(['frame', 'steering', 'throttle', 'brake'])

frame_id = 0

def process_image(image):
    global frame_id
    image_np = np.reshape(np.copy(image.raw_data), (image.height, image.width, 4))
    image_rgb = image_np[:, :, :3]
    
    # Save image
    image_filename = f'dataset/images/frame_{frame_id:05d}.png'
    cv2.imwrite(image_filename, image_rgb)

    # Get current vehicle control state
    control = vehicle.get_control()
    csv_writer.writerow([frame_id, control.steer, control.throttle, control.brake])
    
    frame_id += 1

camera.listen(lambda image: process_image(image))


In [5]:
camera.stop()
camera.destroy()
vehicle.destroy()
control_data.close()
